In [1]:
# Python 2/3 compatibility imports
from __future__ import print_function
from six.moves import input

import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

try:
    from math import pi, tau, dist, fabs, cos
except:  # For Python 2 compatibility
    from math import pi, fabs, cos, sqrt

    tau = 2.0 * pi

    def dist(p, q):
        return sqrt(sum((p_i - q_i) ** 2.0 for p_i, q_i in zip(p, q)))


from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


In [2]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node("move_group_python_interface_tutorial", anonymous=True)

In [3]:
robot = moveit_commander.RobotCommander()

In [4]:
scene = moveit_commander.PlanningSceneInterface()

In [5]:
group_name = "panda_arm"
move_group = moveit_commander.MoveGroupCommander(group_name)

group_name_hand = "panda_hand"
move_group_hand = moveit_commander.MoveGroupCommander(group_name_hand)

In [6]:
display_trajectory_publisher = rospy.Publisher(
    "/move_group/display_planned_path",
    moveit_msgs.msg.DisplayTrajectory,
    queue_size=20,
)

In [7]:
move_group.set_max_velocity_scaling_factor(0.3)
move_group.set_max_acceleration_scaling_factor(0.3)
move_group_hand.set_max_velocity_scaling_factor(0.02)
move_group_hand.set_max_acceleration_scaling_factor(0.02)

In [8]:
# We can get the name of the reference frame for this robot:
planning_frame = move_group.get_planning_frame()
#print("============ Planning frame: %s" % planning_frame)

# We can also print the name of the end-effector link for this group:
eef_link = move_group.get_end_effector_link()
#print("============ End effector link: %s" % eef_link)

# We can get a list of all the groups in the robot:
group_names = robot.get_group_names()
#print("============ Available Planning Groups:", robot.get_group_names())

# Sometimes for debugging it is useful to print the entire state of the
# robot:
#print("============ Printing robot state")
#print(robot.get_current_state())
#print("")

In [23]:
!rostopic pub -1 /franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"


publishing and latching message for 3.0 seconds


In [21]:
# We get the joint values from the group and change some of the values:
def reset():
    
    move_group.set_max_velocity_scaling_factor(0.3)
    move_group.set_max_acceleration_scaling_factor(0.3)
    
    joint_goal = move_group.get_current_joint_values()
    joint_goal[0] = 0 * pi /180
    joint_goal[1] = -48 * pi / 180
    joint_goal[2] = 0 * pi / 180
    joint_goal[3] = -153 * pi / 180
    joint_goal[4] = 0
    joint_goal[5] = 106 * pi / 180
    joint_goal[6] = 44 * pi / 180

    # The go command can be called with joint values, poses, or without any
    # parameters if you have already set the pose or joint target for the group
    move_group.go(joint_goal, wait=True)

    # Calling ``stop()`` ensures that there is no residual movement
    move_group.stop()

In [22]:
reset()

In [24]:
def go_to_button():
    
    move_group.set_max_velocity_scaling_factor(0.3)
    move_group.set_max_acceleration_scaling_factor(0.3)
    
    joint_goal = move_group.get_current_joint_values()
    joint_goal[0] = -65 * pi /180
    joint_goal[1] = -78 * pi / 180
    joint_goal[2] = 91 * pi / 180
    joint_goal[3] = -167 * pi / 180
    joint_goal[4] = 88 * pi / 180
    joint_goal[5] = 103 * pi / 180
    joint_goal[6] = -6 * pi / 180

    # The go command can be called with joint values, poses, or without any
    # parameters if you have already set the pose or joint target for the group
    move_group.go(joint_goal, wait=True)

    # Calling ``stop()`` ensures that there is no residual movement
    move_group.stop()
    
    #######################
    
    

In [25]:
go_to_button()

In [28]:
def grip():
    hand_goal = move_group_hand.get_current_joint_values()
    
    hand_goal[0] = 0
    move_group_hand.go(hand_goal, wait=True)
    
    move_group_hand.stop()
    

In [29]:
def ungrip():
    hand_goal = move_group_hand.get_current_joint_values()
    
    hand_goal[0] = 0.039
    move_group_hand.go(hand_goal, wait=True)
    
    move_group_hand.stop()

In [68]:
def midgrip(dist):
    hand_goal = move_group_hand.get_current_joint_values()
    
    hand_goal[0] = dist
    move_group_hand.go(hand_goal, wait=True)
    
    move_group_hand.stop()

In [16]:
grip()

In [76]:
import wrapper
wrapper.fun()

My stupid wrapper


In [17]:
ungrip()

In [18]:
midgrip()

In [24]:
def move_down_plan(dist=0.27):
    waypoints = []

    wpose = move_group.get_current_pose().pose
    wpose.position.z -= dist  
    waypoints.append(copy.deepcopy(wpose))

    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold

    # Note: We are just planning, not asking move_group to actually move the robot yet:

    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.1,
                                 acceleration_scaling_factor=0.1)
    
    display_trajectory = moveit_msgs.msg.DisplayTrajectory()
    display_trajectory.trajectory_start = robot.get_current_state()
    display_trajectory.trajectory.append(new_plan)
    # Publish
    display_trajectory_publisher.publish(display_trajectory)
    
    return new_plan

In [77]:
import time

In [25]:
move_group.execute(move_down_plan(), wait=True)

True

In [31]:
def push_button():

    ungrip()
    stop_controllers()
    stiffness = [100.,25.,100., 100.,100.,100.,100]
    set_joint_stiffness(stiffness)
    start_controllers()
    
    

    grip()
    
    
    waypoints = []

    wpose = move_group.get_current_pose().pose
    print("z",wpose.position.z)
    wpose.position.z -= 0.016
    waypoints.append(copy.deepcopy(wpose))

    wpose.position.z += 0.05  
    waypoints.append(copy.deepcopy(wpose))

    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold



    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.03,
                                 acceleration_scaling_factor=0.06)
    
    move_group.execute(new_plan, wait=True)
    
    stop_controllers()
    stiffness = [10000.,10000.,10000., 10000.,10000.,10000.,10000]
    set_joint_stiffness(stiffness)
    start_controllers()


In [10]:
from controller_manager_msgs.srv import SwitchController
from controller_manager_msgs.srv import SwitchControllerRequest
switch_controller = rospy.ServiceProxy('/controller_manager/switch_controller', SwitchController)

def stop_controllers():
    req = SwitchControllerRequest()
    req.stop_controllers = ['franka_state_controller','position_joint_trajectory_controller']
    switch_controller(req)
    
def start_controllers():
    req = SwitchControllerRequest()
    req.start_controllers = ['franka_state_controller','position_joint_trajectory_controller']
    switch_controller(req)
    


In [11]:
from franka_msgs.srv import SetJointImpedance
from franka_msgs.srv import SetJointImpedanceRequest
set_joint_impedance = rospy.ServiceProxy('/franka_control/set_joint_impedance', SetJointImpedance)
def set_joint_stiffness(stiffness):
    req = SetJointImpedanceRequest()
    req.joint_stiffness = stiffness
    set_joint_impedance(req)
    


In [45]:
def rigid():
    stop_controllers()
    stiffness = [10000.,10000.,10000., 10000.,10000.,10000.,10000]
    set_joint_stiffness(stiffness)
    start_controllers()

def soft():
    stop_controllers()
    stiffness = [100.,25.,100., 100.,100.,100.,100]
    set_joint_stiffness(stiffness)
    start_controllers()

In [44]:
!rostopic pub -1 /franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"

publishing and latching message for 3.0 seconds


In [160]:
reset()

In [161]:
go_to_button()

In [32]:
push_button()

z 0.215225670063


In [33]:
def recover():
    
    !rostopic pub -1 /franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"
    

In [94]:
joint_goal = move_group.get_current_joint_values()
joint_goal[0] = 0 * pi /180
joint_goal[1] = -79 * pi / 180
joint_goal[2] = 13 * pi / 180
joint_goal[3] = -139 * pi / 180
joint_goal[4] = 0
joint_goal[5] = 96 * pi / 180
joint_goal[6] = 53 * pi / 180
move_group.go(joint_goal, wait=True)


# Calling ``stop()`` ensures that there is no residual movement
move_group.stop()

In [93]:
reset()

In [59]:
def go_to_door():

    #midgrip()
    
    move_group.set_max_velocity_scaling_factor(0.2)
    move_group.set_max_acceleration_scaling_factor(0.2)

    joint_goal = move_group.get_current_joint_values()
    joint_goal[0] = -40 * pi /180
    joint_goal[1] = -41 * pi / 180
    joint_goal[2] = +81 * pi / 180
    joint_goal[3] = -164 * pi / 180
    joint_goal[4] = +65 * pi / 180
    joint_goal[5] = +135 * pi / 180
    joint_goal[6] = +29 * pi / 180
    move_group.go(joint_goal, wait=True)


    # Calling ``stop()`` ensures that there is no residual movement
    move_group.stop()

In [88]:

go_to_door()

In [104]:
midgrip()

In [89]:
def grab_door():

    
    midgrip(0.02)
    
    waypoints = []

    wpose = move_group.get_current_pose().pose
    print("z",wpose.position.z)
    wpose.position.z -= 0.028
    waypoints.append(copy.deepcopy(wpose))
    
    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold



    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.03,
                                 acceleration_scaling_factor=0.06)
    
    move_group.execute(new_plan, wait=True)
    
    #############################3
   
    soft()
    midgrip(0.005)
    
    waypoints = []
    
    wpose = move_group.get_current_pose().pose
    print("z",wpose.position.z)
    wpose.position.z += 0.03  
    waypoints.append(copy.deepcopy(wpose))

    (plan, fraction) = move_group.compute_cartesian_path(
        waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
    )  # jump_threshold



    new_plan=move_group.retime_trajectory(move_group.get_current_state(),
                                 plan,velocity_scaling_factor=0.03,
                                 acceleration_scaling_factor=0.06)
    
    move_group.execute(new_plan, wait=True)
    
    #rigid()

In [90]:
grab_door()

z 0.231796494667
z 0.204254035239


In [85]:
recover()

publishing and latching message for 3.0 seconds


In [86]:
reset()


In [74]:
ungrip()

In [87]:
rigid()

In [61]:
grip()